# Assignment 3 ( Question 4) Know the hatred!

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- all the packages that are imported in the template code

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q4.ipynb`.


In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import sklearn
import torch


In [ ]:
DEVICE = torch.device("cuda:0"  if torch.cuda.is_available() else "cpu")

In [ ]:
DEVICE

device(type='cpu')

In [2]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModel

     |████████████████████████████████| 4.0 MB 5.7 MB/s 
     |████████████████████████████████| 895 kB 35.6 MB/s 
     |████████████████████████████████| 6.6 MB 31.8 MB/s 
     |████████████████████████████████| 596 kB 42.5 MB/s 
     |████████████████████████████████| 77 kB 1.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Dataset

In the training data, the comments are labelled as one or more of the six categories; toxic, severe toxic, obscene, threat, insult and identity hate. This is essentially a multi-label classification problem.

The dataset here is a multi-label classification dataset. To understand multi-label classification datasets, you can refer here: https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/ 

.


Here, we will be using Binary relevance to solve our multi-label classification problem



**Binary Relevance**: This is probably the simplest which treats each label as a separate single classification problems. The key assumption here though, is that there are no correlation among the various labels.

Finally, we will be summing up the accuracies obtained of all labels to obtain the final accuracy


In [ ]:
# loading the dataset

# train_df = pd.read_csv('q4_data/train.csv')
# test_df = pd.read_csv('q4_data/test.csv')

# #dataset understanding
# train_df.sample(5)


In [4]:
# loading the dataset

train_df = pd.read_csv('drive/MyDrive/q4_data/train.csv')
test_df = pd.read_csv('drive/MyDrive/q4_data/test.csv')
test_labels_df = pd.read_csv('drive/MyDrive/q4_data/test_labels.csv')
#dataset understanding
train_df.sample(5)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
43724,74b87f076323d01a,""", 3 November 2013 (UTC)\n\nAgree Israel is a ...",0,0,0,0,0,0
115010,6703f37369327949,Please do NOT deleted the theme song informati...,0,0,0,0,0,0
48725,82490e2e97d9740b,Pics in storagesorry. Will get 'em eventually...,0,0,0,0,0,0
32572,56a80a77b4952633,"""\n Hi, , and Welcome to Wikipedia!\nPlease re...",0,0,0,0,0,0
9613,19783386d2d4be37,"""\n\n Eastern Europe \n\nThe article says: """"F...",0,0,0,0,0,0


In [ ]:
train_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [ ]:
test_df

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [ ]:
test_labels_df.loc[5]['id'] == test_df.loc[5]['id']

True

In [ ]:
test_labels_df

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,-1,-1,-1,-1,-1,-1
153160,fffd7a9a6eb32c16,-1,-1,-1,-1,-1,-1
153161,fffda9e8d6fafa9e,-1,-1,-1,-1,-1,-1
153162,fffe8f1340a79fc2,-1,-1,-1,-1,-1,-1


In [5]:
# let's see the total rows in train, test data and the numbers for the various categories
# cols_target = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print('Total rows in test is {}'.format(len(test_df)))
print('Total rows in train is {}'.format(len(train_df)))
# print(train_df[cols_target].sum())


Total rows in test is 153164
Total rows in train is 159571


In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


In [7]:
train_df['comment_text'] = train_df['comment_text'].map(lambda com : clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com : clean_text(com))

In [ ]:
train_df[:4]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i am s...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i am really not trying to edit war it ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i cannot make any real suggestions on imp...,0,0,0,0,0,0


In [ ]:
test_df[:4]

,id,comment_text
0,00001cee341fdb12,yo bitch ja rule is more succesful then you wi...
1,0000247867823ef7,from rfc the title is fine as it is imo
2,00013b17ad220c46,sources zawe ashton on lapland
3,00017563c3f7919a,if you have a look back at the source the info...


In [8]:
index_names = train_df[ (train_df['toxic'] == -1) & (train_df['severe_toxic'] == -1) & (train_df['obscene'] == -1) & (train_df['threat'] == -1)].index
train_df.drop(index_names, inplace=True)

In [9]:
index_names_test = test_labels_df[ (test_labels_df['toxic'] == -1) & (test_labels_df['severe_toxic'] == -1) & (test_labels_df['obscene'] == -1) & (test_labels_df['threat'] == -1)].index
index_names

Int64Index([], dtype='int64')

In [ ]:
test_labels_df.shape

(153164, 7)

In [ ]:
# index_names = test_labels_df[ (test_labels_df['toxic'] == 1) | (test_labels_df['severe_toxic'] == 1) | (test_labels_df['obscene'] == 1) | (test_labels_df['threat'] == 1)].index
# index_names

In [10]:
#train_df = train_df.drop('char_length',axis=1)
X = train_df.comment_text
test_X = test_df.comment_text
print(X.shape, test_X.shape)


(159571,) (153164,)


In [ ]:
type(test_X)

pandas.core.series.Series

Now, we will be experimenting this classification problem with different classifiers. We will be using pre-trained models inorder to obtain the required embeddings. Here, we will be using BERT and Universal Sentence Encoder embeddings.


Understand how Elmo and USE(Universal Sentence Encoder) works here. You can use any of both to train your classifiers.

To understand how different sentence embeddings work, you can refer to this link: https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial 

Please give it a look before you start working with transfer learning/ sentence embeddings like BERT, Elmo, Ulm-fit, USE.


In [ ]:
!pip install tensorflow_hub

In [ ]:
#loading universal sentence encoder

import tensorflow_hub as hub
# Load pre-trained universal sentence encoder model
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# elmo_embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [ ]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence])
  embedding = embedding.numpy()
  return embedding[0]

In [ ]:
#to obtain bert embeddings ( you need not completely understand how it works)
#you can directly use the function get_bert_embedding to obtain the embedding


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')


def get_bert_embedding(sentence):
  encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embedding[0]


Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
#Obtain X_test and X_train 
X_train_use = []
X_test_use = []
X_train_bert = []
X_test_bert = []

In [ ]:
for one in X:
  X_train_use.append(get_use_embedding(one))
  X_train_bert.append(get_bert_embedding(one[:512]))

for one in test_X:
  X_test_use.append(get_use_embedding(one))
  X_test_bert.append(get_bert_embedding(one))
  

In [ ]:
bert_to_memory_train = []
bert_to_memory_test = []
for i in range(len(X_train_bert)):
  tensor = torch.tensor(X_train_bert[i], dtype=torch.float32, requires_grad=True).cuda()
  np_c = tensor.detach().cpu().nupy()
  bert_to_memory_train.append(np_c)
X_train_bert = np.array(bert_to_memory_train)

In [ ]:
for i in range(len(X_test_bert)):
  tensor = torch.tensor(X_test_bert[i], dtype=torch.float32, requires_grad=True).cuda()
  np_c = tensor.detach().cpu().nupy()
  bert_to_memory_train.append(np_c)
X_test_bert = np.array(bert_to_memory_test)

In [ ]:
X_train_bert = np.loadtxt('drive/MyDrive/train_bert.txt')
X_test_bert = np.loadtxt('drive/MyDrive/test_bert.txt')

In [11]:
X_train_use = np.loadtxt('drive/MyDrive/train_use.txt')
X_test_use = np.loadtxt('drive/MyDrive/test_use.txt')

In [ ]:
X_train_use.shape, X_train_bert.shape

((159571, 768), (159571, 768))

In [ ]:
X_test_bert.shape, X_test_use.shape

((201745, 768), (153164, 512))

In [ ]:
test_df.shape

(153164, 2)

In [16]:
test_labels_df.shape

(63978, 7)

In [12]:
test_labels_df.drop(index_names_test, inplace=True)

In [13]:
test_df_use = pd.DataFrame(X_test_use)

In [14]:
test_df_use.drop(index_names_test, inplace=True)

KeyError: ignored

In [15]:
test_df_use.shape

(63978, 512)

# Classifiers
Here, we use different classifiers for our classification problem. You can directly import classifiers for first two.


*   Naive Bayes classifier
*   Support Vector Machine
*   Multi-layer Perceptron(MLP)

In [ ]:
#calculate the accuracies obtained, u can use sklearn metrics


In [ ]:
# Naive Bayes classifier 

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
!pip install scikit-multilearn

     |████████████████████████████████| 89 kB 5.8 MB/s 


In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
from sklearn.svm import SVC


In [ ]:
# NB_pipeline = Pipeline([
#                 ('tfidf', TfidfVectorizer(stop_words=stop_words, lowercase=False)),
#                 ('clf', OneVsRestClassifier(MultinomialNB(
#                     fit_prior=True, class_prior=None))),
#             ])


In [19]:
categories = train_df.columns[2:].tolist()
categories

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
train_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i am s...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i am really not trying to edit war it ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i cannot make any real suggestions on imp...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,and for the second time of asking when your vi...,0,0,0,0,0,0
159567,ffea4adeee384e90,you should be ashamed of yourself that is a ho...,0,0,0,0,0,0
159568,ffee36eab5c267c9,spitzer umm theres no actual article for prost...,0,0,0,0,0,0
159569,fff125370e4aaaf3,and it looks like it was actually you who put ...,0,0,0,0,0,0


In [ ]:
X_train_bert

array([[-0.15117911,  0.92516446,  0.5815081 , ...,  0.2849282 ,
        -0.34590417,  0.28080872],
       [-0.08058564,  1.41396666,  1.21551061, ...,  0.10520222,
         0.44793549,  0.36692765],
       [-0.12137693,  1.05800498,  0.95669776, ...,  1.07018721,
        -0.48925155,  0.37755999],
       ...,
       [ 0.44760653,  0.79723346,  0.10681202, ...,  0.01412262,
        -0.52592731, -0.21340628],
       [-0.28964734,  0.20100908,  0.65541101, ...,  0.03621923,
        -0.194758  ,  0.08033364],
       [ 0.06439633,  1.02616167,  0.63475096, ..., -0.04997476,
        -0.93604249,  0.25031981]])

In [ ]:
# for category in categories:
#     print('... Processing {}'.format(category))
#     # train the model using X_dtm & y
#     NB_pipeline.fit(pd.DataFrame(X_train_bert), train_df[category])
#     # compute the testing accuracy
#     prediction = NB_pipeline.predict(pd.DataFrame(X_test_bert))
#     print('Test accuracy is {}'.format(accuracy_score(test_df[category], prediction)))

# USE embedding

In [ ]:
X_train_use[:, :512].shape, X_test_use.shape

In [ ]:
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(X_train_use[:, :512], train_df[categories])

# predict
predictions = classifier.predict(test_df_use)

In [ ]:
predictions.shape

(63978, 6)

In [ ]:
test_labels_df[categories].iloc[:,[0]].shape

(63978, 1)

In [ ]:
test_labels_df[categories].shape

(63978, 6)

In [ ]:
for i in range(len(categories)):
  actual = test_labels_df[categories].iloc[:,i]
  predicted = predictions[:, i]

  actual = actual.to_numpy().flatten()
  predicted = predicted.toarray().flatten()
  print('for category -', categories[i], ': Accuracy: ',accuracy_score(actual, predicted))

for category - toxic : Accuracy:  0.9048110287911469
for category - toxic : f1_score:  0.0
for category - severe_toxic : Accuracy:  0.9942636531307637
for category - severe_toxic : f1_score:  0.0
for category - obscene : Accuracy:  0.9423082934758823
for category - obscene : f1_score:  0.0
for category - threat : Accuracy:  0.9967019913095126
for category - threat : f1_score:  0.0
for category - insult : Accuracy:  0.9464347119322267
for category - insult : f1_score:  0.0
for category - identity_hate : Accuracy:  0.9888711744662227
for category - identity_hate : f1_score:  0.0


In [ ]:
#Support Vector Machines

In [ ]:
classifier = BinaryRelevance(SVC())

# train
classifier.fit(X_train_use[:, :512], train_df[categories])

# predict
predictions = classifier.predict(test_df_use)

In [ ]:
for i in range(len(categories)):
  actual = test_labels_df[categories].iloc[:,i]
  predicted = predictions[:, i]

  actual = actual.to_numpy().flatten()
  predicted = predicted.toarray().flatten()
  print('for category -', categories[i], ': Accuracy: ',accuracy_score(actual, predicted))

#Bert Embedding

In [ ]:
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(X_train_bert, train_df[categories])

# predict
predictions = classifier.predict(X_test_bert)

In [ ]:
predictions.shape

(201745, 6)

In [ ]:
test_labels_df[categories].shape

(153164, 6)

In [ ]:
test_labels_df[test_labels_df[categories] == -1]

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...
153159,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
153160,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
153161,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
153162,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [ ]:
max_len_test = min(predictions.shape[0], test_labels_df[categories].shape[0])

In [ ]:
test_labels_df.shape

(153164, 7)

In [ ]:
# test_labels_df[categories][:max_len_test, [0]].shape

In [ ]:
# test_labels_df[categories].iloc[:max_len_test, :].shape

In [ ]:
# np.argmax(test_labels_df[categories].iloc[:max_len_test, 0], axis=1)

In [ ]:
# predictions[:,[0]][0]

In [ ]:
# print(classification_report(predictions[:max_len_test], test_labels_df[categories][:max_len_test]))

In [ ]:
#Support Vector Machines

Which classifier of the three performed better, and why do you think so ?

Ans: SVM performed better than other 2 models in terms of accuracy. The main assumption in Naive Bayes is taht all the features presenta are independent of each other. Whereas SVM tries to find some relation between features.

The SVM can be used and works very well in Multilabel classification with techniques like One vs rest and One vs One. Where as the MLP works very good foe sinle class classification. In case of sklearn MLP we were not able to apply different activation in different layers, the performance would have increased. This helped the SVM to get slightly better accuracy than sklearn MLP.


#Multi-layer Perceptron

For Multi-layer Perceptron, first you will be implementing your own articial Neural network, and then compare it with sklearn's MLP layer.


Implement your own artificial Neural Network (MLP)
Few Steps to be followed(for your reference):

*   Initialise the weights randomly
*   Decide upon the number of layers you wanna have and number of neurons associated in each layer. ( Have a clear idea of what will be the size of the weight matrices).
*   Write the code for forward and backward propogations
*   Decide upon the loss function
*   Train the model
*   Predict the labels after training the model




In [26]:
import numpy as np
# you can remove returns statements if u think they are not required
# you can also add functions if needed
# creating a (let's just take a generalized one, actually) L-layer NN all relu except o/u which is sigmoid/softmax 
def sigmoid(x):
  return 1/(1+np.exp(-x))

def initialise_weights(L_list):
  params = {}
  for i in range(1,len(L_list)):
    params[f'w_{i}'] = np.random.randn(L_list[i], L_list[i-1])
    params[f'b_{i}'] = np.zeros((L_list[i],1))
  
  return params  

def loss_function(A_l, y):
  return sum(sum(-y*np.log(A_l)))


def forward_layer(X, params):
  temp = {}
  a = X
  temp['a_0'] = a
  l = int(len(params)/2)
  for i in range(1, l):
    z = np.dot(params[f'w_{i}']) + params[f'b_{i}']
    a = np.maximum(z, 0) #ReLU activation
    temp[f'z_{i}'] = z
    temp[f'a_{i}'] = a
  z = np.dot(params[f'w_{l}']) + params[f'b_{l}']
  temp[f'z_{l}'] = z
  a = sigmoid(z)
  temp[f'a_{l}'] = a
  return a, temp


def backward_propogation(y, a, params, temp):
  grads = {}
  l = int(len(params)/2)
  dA, dZ = [], []
  dA.append(np.sum(-y/a, axis = 0))
  dZ.append(dA[-1]*a*(1-a))   #sigmoid differentiation
  m = y.shape[1]

  for i in reversed(range(1, l)):
    dA.append(np.dot(params[f'w_{i+1}'].T,dZ[-1]))
    dZ_ = np.array(dA[-1], copy=True)
    dZ_[temp[f'z_{i}'] <= 0] = 0
    dZ.append(dZ_)
  
  for i in range(1,l+1):
    db = (1/m) * np.sum(dZ[-i], axis = 1, keepdims=True)
    dw = (1/m) * np.dot(dZ[-i], temp[f'a_{i-1}'].T)
    assert db.shape == params[f'b_{i}'].shape
    assert dw.shape == params[f'w_{i}'].shape
    grads[f'db_{i}'] = db
    grads[f'dw_{i}'] = dw
    
  return grads


In [ ]:
# def train(X, y, L_list, epoch = 3000, alpha = 0.1):
#     params = initialise_weights(L_list)
    
#     for i in range(epoch):
#         a_l, temp = forward_layer(X, params)
#         loss = loss_function(a_l, params)
#         grads = backward_propogation(y, a_l, params, temp)
        
#         for j in range(1,len(L_list)):
#             params[f'b_{j}'] -= alpha * grads[f'db_{i}']
#             params[f'w_{j}'] -= alpha * grads[f'dw_{i}']
        
#         if i%100 == 0:
#             print(f"Loss value after {i} iterations:\t{loss}")
  

In [27]:
#report accuracies
#here, you need to predict for each of the label as we have trained classifiers in such a way

def train(X, y, L_list, epoch = 3000, alpha = 0.01):
    params = initialise_weights(L_list)
    
    for i in range(epoch):
        a_l, temp = forward_layer(X, params)
        loss = loss_function(a_l, y)
        grads = backward_propogation(y, a_l, params, temp)
        
        for j in range(1,len(L_list)):
            params[f'b_{j}'] = params[f'b_{j}'] - alpha * grads[f'db_{j}']
            params[f'w_{j}'] = params[f'w_{j}'] - alpha * grads[f'dw_{j}']
        
        if i%500 == 0:
            print(f"Loss value after {i} iterations:\t{loss}")
            
    return params
  
def predict(X_test, params):
    a, temp = forward_layer(X_test, params)
    return a


#USE Embedding

In [28]:
for category in categories[:1]:
  model = train(X_train_use[:, :512].T, train_df[category].T, [512, 300, 100, 50, 1])
  predictions = predict(X_train_use, model)
  print(predictions)


ValueError: ignored

In [ ]:
# for i in range(len(categories)):
#   # actual = test_labels_df[categories].iloc[:,i]
#   # predicted = predictions[:, i]

#   # actual = actual.to_numpy().flatten()
#   # predicted = predicted.toarray().flatten()
#   print('for category -', categories[i], ': Accuracy: ',accuracy_score(actual, predicted))

In [ ]:
#report accuracies on the test dataset 





Now, compare your implemented MLP with the sklearn MLP layer.

In [ ]:
#report accuracies both on train dataset and test dataset

from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(512, 300, 100, 50, 1), random_state=1, max_iter=300).fit(X_train_use[:, :512], train_df[categories])
predictions = model.predict(X_test_use)

In [ ]:
for i in range(len(categories)):
  actual = test_labels_df[categories].iloc[:,i]
  predicted = predictions[:, i]

  actual = actual.to_numpy().flatten()
  predicted = predicted.toarray().flatten()
  print('for category -', categories[i], ': Accuracy: ',accuracy_score(actual, predicted))

Which MLP performed better? 
Understand and Analyse.

The MLP built by me gave better accuracy. The reason can be that I ahve used different activation function in different layers. In Sklearn MLP the activation funvtion remained same in all layers
Because our MLP is just a vanilla MLP, sklearn's MLP would have done better in an some environment. Sklearn MLP would have worked better in the data where same activation in all layers would have been suffice.
So, while SkLearn's MLP performed better, our prediction measurement was poor.